# How to Build AI Agents wey get Persistent Memory wit Cognee

Dis notebook go show how to take build smart AI agents wey sabi memory well well using [**cognee**](https://www.cognee.ai/) - na open source AI memory wey join knowledge graphs, semantic search, and session management to create AI systems wey sabi context.

## 🎯 Wetin You Go Learn

By the time you finish dis tutorial, you go sabi how to:
- **Build Knowledge Graphs wey dey Backed by Embeddings**: Change text wey no get structure into structured knowledge wey you fit query
- **Implement Session Memory**: Create multi-turn conversations wey go dey keep context automatically
- **Keep Conversations for Future**: Store important interactions for long-term memory if you want am
- **Query wit Natural Language**: Use historical context for new conversations
- **See Memory Visuals**: Check how your agent's knowledge graph dey connect


## 🏗️ Wetin You Go Build

For dis tutorial, we go create one **Coding Assistant** wey get memory wey no dey loss, wey fit do di following:

### 1. **Build Knowledge Base**
   - Go fit collect developer profile and di kain expertise wey dem get
   - Go sabi process Python programming principles and di best ways to take use am
   - Go dey store old conversations wey don happen between developers and AI assistants

### 2. **Session-Aware Conversations**
   - Go fit remember context for di same session even if na multiple questions
   - Go dey automatically save each question/answer pair so e go dey easy to find later
   - Go dey give correct and meaningful response based on di conversation wey don happen before

### 3. **Long-term Memory**
   - Go dey keep important conversations for long-term memory
   - Go fit bring out relevant memories from di knowledge base and past sessions to help for new interactions
   - Go dey build di knowledge base small small so e go dey better as time dey go

### 4. **Intelligent Memory Retrieval**
   - Go use graph-aware semantic search to find di correct information from all di knowledge wey e don store
   - Go fit filter di search by di kain data group (like developer info or principles)
   - Go dey combine different data sources to give complete and correct answers


## 📋 Wetin You Need & How to Set Up

### System Requirements

Before you start, make sure say you get:

1. **Python Environment**
   - Python 3.9 or higher
   - Virtual environment (e good make you use am)

2. **Redis Cache** (You go need am for Session Management)
   - Local Redis: `docker run -d -p 6379:6379 redis`
   - Or make you use managed Redis service

3. **LLM API Access**
   - OpenAI API key or other providers (check [documentation](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Database Configuration**
   - By default, you no need configure anything. Cognee dey use file-based databases (LanceDB and Kuzu)
   - If you wan, you fit set up Azure AI Search as vector store (check [documentation](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Environment Configuration

Create `.env` file for your project directory wey go get these variables:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Understanding Cognee Memory Architecture

### How Cognee Dey Work

Cognee get one kind beta memory system wey pass just ordinary key-value storage:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Di Main Parts:

1. **Knowledge Graph**: E dey store entities, relationships, and semantic connections
2. **Vector Embeddings**: E dey make semantic search possible for all di information wey dem store
3. **Session Cache**: E dey keep conversation context for inside and between sessions
4. **NodeSets**: E dey arrange data into logical categories so retrieval go dey easy

### Di Kain Memory We Go Talk About for Dis Tutorial:

- **Persistent Memory**: Na long-term storage for di knowledge graph
- **Session Memory**: Na temporary conversation context wey dey for Redis cache
- **Semantic Memory**: Na vector-based similarity search wey dey work for all di data


## 📦 Install Di Package Wey You Need

Install Cognee wey get Redis support for session management:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Set Up Environment and Load Libraries

Make sure say:
1. Redis dey run (e.g., if you wan use Docker: `docker run -d -p 6379:6379 redis`)
2. You don set environment variables before you import cache modules
3. If e need, restart di kernel and run di cells one by one for correct order

Di cell wey dey follow go do di following:
1. Load environment variables from `.env`
2. Configure Cognee wit your LLM settings
3. Enable caching for session management
4. Check say all di components connect well well


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Configure Storage Directories

Cognee dey use two diffren directories for di work wey e dey do:  
- **Data Root**: Na here e dey store di documents wey e don collect and di data wey e don process.  
- **System Root**: Na here e dey keep di knowledge graph database and di system metadata.  

We go create separate directories for dis tutorial like dis:  


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Reset Memory State

Before we go start to build our memory system, make we make sure say we dey start afresh.

> 💡 **Tip**: You fit skip dis step if you wan keep di memories wey dey already from di runs wey you don do before when you go use dis notebook later.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Part 1: How to Build Di Knowledge Base

### Data We Go Use for Our Developer Assistant

We go collect three kain data to build beta knowledge base:

1. **Developer Profile**: Di person skill and wetin dem sabi for tech
2. **Python Best Practices**: Di Zen of Python wit practical guidelines
3. **Historical Conversations**: Old Q&A wey don happen between developers and AI assistants

Dis kain data go help our agent to:
- Sabi di user technical level
- Use best practices for di advice wey e go give
- Learn from di past interactions wey don work well


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Process Data into Knowledge Graph

Now we go change our raw text into structured memory. Dis process:

1. **Add data to NodeSets**: E go arrange information into correct categories
   - `developer_data`: Developer profile and wetin dem talk
   - `principles_data`: Python best practices and guidelines

2. **Run Cognify Pipeline**: E go extract entities, relationships, and create embeddings
   - E go find key concepts
   - E go connect related information semantically
   - E go generate vector embeddings

Dis one fit take small time as LLM dey process di text and dey build di graph structure:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 See Di Knowledge Graph

Make we check how di knowledge graph be. Di visualization go show:
- **Nodes**: Entities wey dem comot from di text (concepts, technologies, people)
- **Edges**: Di relationships and connections wey dey between di entities
- **Clusters**: Related concepts wey dem group based on semantic similarity

Open di HTML file wey dem generate for your browser to fit explore di graph well well:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Add Memory wit Memify

Di `memify()` function dey check di knowledge graph and e dey create smart rules about di data. Dis process:
- E dey find patterns and di best way to do things
- E dey make guidelines wey you fit use based on di content
- E dey connect different knowledge areas together

Dis rules go help di agent sabi better how to answer questions. If you capture second visualization, e go help you see how di graph don get more connections after e don enrich am.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Part 2: Intelligent Memory Retrieval

### Demonstration 1: Cross-Document Knowledge Integration

Now wey we don build di knowledge graph, make we test how Cognee dey join information from different sources to fit answer complex questions.

Di first query go show:
- **Semantic understanding**: How e dey find di correct concepts even if dem no talk am directly
- **Cross-referencing**: How e dey join developer profile with Python principles
- **Contextual reasoning**: How e dey use best practices for specific implementations

### Demonstration 2: Filtered Search with NodeSets

Di second query go show how you fit target specific parts of di knowledge graph:
- E dey use `node_name` parameter to search only inside `principles_data`
- E dey give focused answers from one particular knowledge area
- E dey useful when you need information wey dey specific to one domain


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Part 3: Session Management Setup

### How to Make Conversation Memory Work

Session management dey very important to keep context for plenty interactions. For here we go:

1. **Start User Context**: Create or find user profile wey go help track session
2. **Set Cache Engine**: Connect am to Redis to store conversation history
3. **Turn On Session Variables**: Arrange context variables wey go dey stay for all queries

> ⚠️ **Important**: This one need make Redis dey run and `CACHING=true` for your environment


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Helper Function: View Session History

Dis function go help us check di tok wey don dey store for Redis. E dey useful for:
- Debug di way session dey work
- Make sure say di tok dem dey save well
- Understand di kind context wey di agent fit use


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Session 1: Async Support Lab — First Question

Start the `async-support-lab` session by ask how person fit use telemetry-friendly asyncio patterns for one big web scraper. Di graph don already sabi asyncio, aiohttp, and monitoring practices, so di answer suppose match wetin dem don talk before but e go still fit di new question.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Check Session 1 Memory After Di Fes Tok

If you run `show_history(session_1)` just afta di first question, e go show say Cognee don put di prompt and di answer inside Redis. You go see one entry wey get di concurrency guidance.


In [ ]:
await show_history(session_1)

## Session 1: Follow-up on Data Models

Next we go ask, "Wen I go fit choose dataclasses or Pydantic?" we go use di same session id. Cognee go join di Python principles plus di FastAPI talk wey we don do before to give better advice—show say context dey continue inside one session wey get name.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Confirm Session 1 History Get Both Turns

Another `show_history(session_1)` call go show two Q&A entries. Dis one match di Mem0 lab "memory replay" step and e show say extra turns dey add to di same transcript.


In [ ]:
await show_history(session_1)

## Session 2: Design Review Thread — Fresh Session

To show say threads dey separate, we go start `design-review-session` and ask for logging guidance for incident reviews. Even though di knowledge base wey dey under dey di same, di new session id go keep transcripts separate.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Review Session 2 History

`show_history(session_2)` suppose show only di design-review prompt/response pair. Compare am wit Session 1 to show how Cognee dey keep separate transcripts but still dey use di shared knowledge graph.


In [ ]:
await show_history(session_2)

## Summary

Congrats! You don give your coding assistant real long-term memory layer wey dey powered by Cognee.

For dis tutorial, you carry raw developer content (code, docs, chats) turn am into graph + vector memory wey your agent fit search, reason on top, and dey improve steady.

Wetin You Don Learn

1. **From raw text to AI memory**: How Cognee dey take unstructured data turn am into smart, searchable memory using combined vector + knowledge graph architecture.

2. **Graph enrichment with memify**: How you fit go beyond just creating graph and use memify take add extra facts and better relationships on top your graph wey don already dey.

3. **Multiple search strategies**: How to query memory with different search types (graph-aware Q&A, RAG-style completion, insights, raw chunks, code search, etc.) based on wetin your agent need.

4. **Visual exploration**: How to check and debug wetin Cognee don build using graph visualizations and the Cognee UI, so you fit see how knowledge take dey structured.

5. **Session-aware memory**: How to join per-session context with persistent semantic memory so agents fit remember across runs without leaking information between users.


## Key Takeaways
1. Memory as Knowledge Graph wey dey use Embeddings

   - **Beta understanding**: Cognee dey join vector store and graph store so your data go dey searchable by meaning and e go still connect by relationships. Cognee dey use file-based databases by default (LanceDB for vector-, Kuzu for graph database).

   - **Relationship-aware retrieval**: Answers no go just base on “similar text,” but e go also consider how entities take relate.

   - **Living memory**: The memory layer dey evolve, grow, and e go still dey queryable as one connected graph.

2. Search & Reasoning Modes
   - **Hybrid retrieval**: Search dey mix vector similarity, graph structure, and LLM reasoning, from raw chunk lookup to graph-aware question answering.

   - **Fit the mode to the job**: Use completion-style modes if you want natural language answers, and chunk/summary/graph modes if your agent need raw context or wan drive im own reasoning.

3. Personalized, Session-Aware Agents
   - **Session context + long-term memory**: Cognee dey keep short-term “thread” context separate from long-lived, user- or org-level memory.

## Real-World Applications

1. **Vertical AI Agents**

   Use the pattern wey dey this notebook to power domain-smart copilots wey go dey on top of Cognee as their retrieval and reasoning core:

- **Developer copilots**: Code review, incident analysis, and architecture assistants wey fit waka through code, APIs, design docs, and tickets as one memory graph.

- **Customer-facing copilots**: Support or success agents wey dey pull from product docs, FAQs, CRM notes, and past tickets with graph-aware retrieval and cited answers.

- **Internal expert copilots**: Policy, legal, or security assistants wey dey reason over interconnected rules, guidelines, and historical decisions instead of isolated PDFs.

   Cognee dey positioned as persistent, accurate memory for AI agents, wey dey provide living knowledge graph wey fit dey behind your agent and replace ad-hoc combinations of vector stores and custom graph code.

2. **Unifying Data Silos into One Memory**

   This same approach go help you build one unified memory layer across scattered sources:

- **From silos to one graph**: Ingest structured (e.g., databases) and unstructured data (e.g., docs, chats) into one single graph wey dey backed by embeddings, instead of separate indices for each system.

- **Cross-source reasoning with citations**: Run multi-step reasoning over everything—“join” logs, metrics, and docs via the graph—and still return grounded answers with provenance.

- **Knowledge hubs**: For domains like banking or education, Cognee don already dey used to unify PDFs, internal systems, and app data into one knowledge graph with vectors so agents fit answer questions with precise, cited context.

## Next Steps

You don implement the core memory loop. These na natural extensions wey you fit try on your own (see [Cognee documentation](https://docs.cognee.ai/) for details):

1. **Experiment with temporal awareness**: Turn on temporal cognify to extract events and timestamps from text.

2. **Introduce ontology-driven reasoning**: Define an OWL ontology for your domain. Use Cognee’s ontology support so extracted entities and relations go dey grounded in that schema, wey go improve graph quality and domain-specific answers.

3. **Add a feedback loop**: Make Cognee dey adjust graph edge weights from real user feedback, so retrieval go dey improve over time instead of staying static.

4. **Tune for personalization & session behavior**: Use user IDs, tenants, and datasets to give each person or team their own view over the shared memory engine.

5. **Scale out to more complex agents**: Plug Cognee into agent frameworks to build multi-agent systems wey all go dey share the same memory layer. *Microsoft Agent Framework x Cognee plugin dey come soon.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dis dokyument don use AI translation service [Co-op Translator](https://github.com/Azure/co-op-translator) do di translation. Even as we dey try make am correct, abeg make you sabi say machine translation fit get mistake or no dey accurate well. Di original dokyument wey dey for im native language na di main source wey you go trust. For important information, e better make professional human translation dey use. We no go fit take blame for any misunderstanding or wrong interpretation wey fit happen because you use dis translation.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
